# Gradienty w PyTorch
W tej sekcji omawiamy implementację spadku gradientowego w PyTorch przy użyciu modułu <a href='https://pytorch.org/docs/stable/autograd.html'>`autograd`</a>. Wykorzystamy narzędzia:

* <a href='https://pytorch.org/docs/stable/autograd.html#torch.autograd.backward'>`torch.autograd.backward()`</a>
* <a href='https://pytorch.org/docs/stable/autograd.html#torch.autograd.grad'>`torch.autograd.grad()`</a>


Na wstępie musimy przypomnieć krótkio czym są te rzeczy:

### Funkcje aktywacji

* **Skokowa (Heaviside)** – bardzo prosta, zwraca `0` poniżej progu i `1` powyżej. Historyczna, ale nie nadaje się do trenowania sieci (brak pochodnej).
* **Sigmoidalna (σ)** – odwzorowuje wejście w zakres `(0,1)`. Używana jako model „prawdopodobieństwa”. Dobrze gładka, ale przy dużych wartościach wejściowych gradient zanika.

---

### Kodowanie one-hot

Reprezentacja kategorii jako wektora, w którym jedna pozycja ma wartość `1`, a pozostałe `0`.
Np. klasa „kot” przy trzech klasach: `[1,0,0]`.

---

### Maksymalne prawdopodobieństwo (argmax)

W klasyfikacji model zwraca rozkład prawdopodobieństwa po klasach (np. przez **softmax**). Klasa przewidywana to ta o największym prawdopodobieństwie (`argmax`).

---

### Entropia krzyżowa

Miara różnicy między **rozkładem przewidywanym** a **rozkładem rzeczywistym**.

* **Binary cross-entropy** – dla problemów 0/1.
* **Categorical cross-entropy** – dla wielu klas (porównuje przewidywany rozkład softmax z etykietą one-hot).
  Niższa wartość oznacza lepsze dopasowanie.

---

### Propagacja wsteczna (backprop)

Algorytm wyznaczania gradientów w sieci neuronowej:

1. Obliczamy błąd (loss).
2. Rozchodzimy ten błąd wstecz przez warstwy, używając reguły łańcuchowej dla pochodnych.
3. Używamy gradientów do aktualizacji wag (np. w metodzie SGD).

---

👉 Te pojęcia tworzą bazę do zrozumienia, jak PyTorch `autograd` automatycznie śledzi operacje na tensorach, liczy pochodne i pozwala na efektywne uczenie modeli.


<div class="alert alert-info"><h3>Dodatkowe materiały:</h3>
<strong><a href='https://pytorch.org/docs/stable/notes/autograd.html'>Notatki PyTorch:</a></strong>&nbsp;&nbsp;<font color=black>Mechanika Autograd</font></div>


## Autograd – automatyczne różniczkowanie
We wcześniejszych częściach tworzyliśmy tensory i wykonywaliśmy na nich różne operacje, ale nie zapisywaliśmy ich sekwencji ani nie wyznaczaliśmy pochodnej gotowej funkcji.

W tej sekcji wprowadzimy pojęcie <em>dynamicznego grafu obliczeniowego</em>, który składa się ze wszystkich obiektów typu <em>Tensor</em> w sieci oraz <em>funkcji</em>, które je utworzyły. Zauważ, że jedynie tensory wejściowe tworzone przez nas samych nie mają skojarzonych obiektów Function.

Pakiet PyTorch <a href='https://pytorch.org/docs/stable/autograd.html'><strong><tt>autograd</tt></strong></a> zapewnia automatyczne różniczkowanie dla wszystkich operacji na tensorach. Dzieje się tak, ponieważ operacje stają się atrybutami samych tensorów. Gdy atrybut tensora <tt>.requires_grad</tt> ustawimy na True, tensor zaczyna śledzić wszystkie wykonywane na nim działania. Po zakończeniu sekwencji operacji możemy wywołać <tt>.backward()</tt>, aby obliczyć wszystkie gradienty automatycznie. Gradient tensora zostanie zsumowany w jego atrybucie <tt>.grad</tt>.

Zobaczmy to w praktyce.


## Propagacja wsteczna w jednym kroku
Zaczniemy od zastosowania pojedynczej funkcji wielomianowej $y = f(x)$ do tensora $x$. Następnie wykonamy propagację wsteczną i wypiszemy gradient $\frac {dy} {dx}$.

$\begin{split}Function:\quad y &= 2x^4 + x^3 + 3x^2 + 5x + 1 \\
Derivative:\quad y' &= 8x^3 + 3x^2 + 6x + 5\end{split}$

#### Krok 1. Wykonaj standardowe importy


In [1]:
import torch

#### Krok 2. Utwórz tensor z ustawionym <tt>requires_grad</tt> na True
Dzięki temu tensor zacznie śledzić obliczenia.


In [2]:
x = torch.tensor(2.0, requires_grad=True)

In [3]:
x.grad

#### Krok 3. Zdefiniuj funkcję


In [4]:
y = 2*x**4 + x**3 + 3*x**2 + 5*x + 1

print(y)

tensor(63., grad_fn=<AddBackward0>)


Ponieważ $y$ powstało w wyniku operacji, ma skojarzoną funkcję gradientu dostępną jako <tt>y.grad_fn</tt>.<br>
Obliczenie $y$ przebiega następująco:<br>

$\quad y=2(2)^4+(2)^3+3(2)^2+5(2)+1 = 32+8+12+10+1 = 63$

To wartość $y$ dla $x=2$.

#### Krok 4. Propagacja wsteczna


In [5]:
y.backward()

#### Krok 5. Wyświetl uzyskany gradient


In [6]:
print(x.grad)

tensor(93.)


Zauważ, że <tt>x.grad</tt> jest atrybutem tensora $x$, dlatego nie używamy nawiasów. Obliczenie ma postać<br>

$\quad y'=8(2)^3+3(2)^2+6(2)+5 = 64+12+12+5 = 93$

To nachylenie wielomianu w punkcie $(2,63)$.

## Propagacja wsteczna w wielu krokach
Teraz wykonajmy coś bardziej złożonego, z warstwami $y$ i $z$ pomiędzy $x$ a warstwą wyjściową $out$.
#### 1. Utwórz tensor


In [7]:
x = torch.tensor([[1.,2,3],[3,2,1]], requires_grad=True)
print(x)

tensor([[1., 2., 3.],
        [3., 2., 1.]], requires_grad=True)


#### 2. Utwórz pierwszą warstwę z $y = 3x+2$


In [8]:
y = 3*x + 2
print(y)

tensor([[ 5.,  8., 11.],
        [11.,  8.,  5.]], grad_fn=<AddBackward0>)


#### 3. Utwórz drugą warstwę z $z = 2y^2$


In [9]:
z = 2*y**2
print(z)

tensor([[ 50., 128., 242.],
        [242., 128.,  50.]], grad_fn=<MulBackward0>)


#### 4. Ustaw wyjście jako średnią macierzy


In [10]:
out = z.mean()
print(out)

tensor(140., grad_fn=<MeanBackward0>)


#### 5. Wykonaj propagację wsteczną, aby znaleźć gradient $x$ względem <tt>out</tt>
(Jeśli widzisz ten skrót po raz pierwszy, w.r.t. oznacza <em>with respect to</em>, czyli „względem”.)


In [11]:
out.backward()
print(x.grad)

tensor([[10., 16., 22.],
        [22., 16., 10.]])


Powinieneś zobaczyć macierz 2x3. Jeśli nazwiemy końcowy tensor <tt>out</tt> jako "$o$", wówczas możemy obliczyć pochodną cząstkową $o$ względem $x_i$ następująco:<br>

$o = \frac {1} {6}\sum_{i=1}^{6} z_i$<br>

$z_i = 2(y_i)^2 = 2(3x_i+2)^2$<br>

Aby wyznaczyć pochodną $z_i$, korzystamy z <a href='https://en.wikipedia.org/wiki/Chain_rule'>reguły łańcuchowej</a>, według której pochodna $f(g(x)) = f'(g(x))g'(x)$.<br>

W naszym przypadku<br>

$\begin{split} f(g(x)) &= 2(g(x))^2, \quad &f'(g(x)) = 4g(x) \\
g(x) &= 3x+2, &g'(x) = 3 \\
\frac {dz} {dx} &= 4g(x)\times 3 &= 12(3x+2) \end{split}$

Zatem<br>

$\frac{\partial o}{\partial x_i} = \frac{1}{6}\times 12(3x+2)$<br>

$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = 2(3(1)+2) = 10$

$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=2} = 2(3(2)+2) = 16$

$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=3} = 2(3(3)+2) = 22$


## Wyłącz śledzenie
Zdarzają się sytuacje, w których nie chcemy lub nie musimy śledzić historii obliczeń.

Możesz ponownie ustawić atrybut tensora <tt>requires_grad</tt> w miejscu, używając `.requires_grad_(True)` (lub False) w razie potrzeby.

Podczas ewaluacji często warto owinąć operacje blokiem `with torch.no_grad():`

Rzadziej stosowaną metodą jest wywołanie `.detach()` na tensorze, aby uniemożliwić śledzenie przyszłych obliczeń. To przydatne podczas klonowania tensora. Nam też sie przyda w paru miejscach.
